# Prompt selection and testing

This notebook chooses the most appropriate prompt and prompt structure for the OCR correction. 

In [21]:
#import config  # Import your config.py file this contains you openai api key
import pandas as pd
import numpy as np
import os
from llm_comparison_toolkit import RateLimiter, get_response_openai, get_response_anthropic,  create_config_dict_func, compare_request_configurations, generate_model_configs
from evaluate import load
from evaluation_funcs import evaluate_correction_performance, evaluate_correction_performance_folders, get_metric_error_reduction
import seaborn as sns
import matplotlib.pyplot as plt
from helper_functions import files_to_df_func, evaluate_ner, calculate_entity_similarity, repeat_prompt_experiment
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re


dev_data_folder = 'data/dev_data'
dev_transcripts = os.path.join(dev_data_folder, 'dev_data_transcript')
dev_raw_ocr_folder =  os.path.join(dev_data_folder,'dev_raw_ocr' )
dev_system_message_folder = os.path.join(dev_data_folder,'dev_system_message_variants' )

#load the dev and test sets for prompt development and selection
dev_data_df = pd.read_csv(os.path.join(dev_data_folder,'dev_data_raw.csv'))


#for saving data to be used in the analysis
if not os.path.exists('data/analysis'):
    os.makedirs('data/analysis')

# Explore different system prompts

This explores a range of system prompt to find the one that appears to work the best, we use gpt4 as the baseline model.

Although there is no comparison with all models we do test gpt3.5 gpt4, clause haiku and claude opus, in addition we put the prompt in the system message and the prompt after the text becuase this may affect the quality of the result. When the prompt is after the text the response has "nosm_" (no system message) appended to the file name

In [3]:
#Create a modular set of system messages that can be combined in different ways
basic_prompt = "Please recover the text from the corrupted OCR."
expertise_prompt = "You are an expert in post-OCR correction of documents."
recover_prompt = "Using the context available from the text please recover the most likely original text from the corrupted OCR."
publication_context_prompt = "The text is from an english newspaper in the 1800's."
text_context_prompt = "The text may be an advert or article and may be missing the beggining or end."
additional_instructions_prompt = "Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."

#combine all the message parts into a variety of system messages, a tuple is used where 0 is the name of the message and 1 is the message itself
#N.B. This is not and exhaustive combination as that would be very expensive and likley not yield significantly better results
system_messages_list = [
('basic_prompt', basic_prompt),
('expert_basic_prompt', expertise_prompt + ' '+ basic_prompt),
('expert_recover_prompt', expertise_prompt + ' '+ recover_prompt),
('expert_recover_publication_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt),
('expert_recover_text_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + text_context_prompt),
('expert_recover_publication_text_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt + ' ' + text_context_prompt),
('expert_recover_instructions_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + additional_instructions_prompt),
('full_context_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt + ' ' + text_context_prompt+ ' ' + additional_instructions_prompt)
]

The below function is used to make the creation of the config dictionaries for the test more compact and increase readability

In [4]:
def create_message_test_configs(system_messages_list, get_response_func, engine):
    message_test_configs = []
    for iter_system_message in system_messages_list:
        message_test_configs.append(
            create_config_dict_func(
                get_response_func=get_response_func,
                rate_limiter=RateLimiter(50000),
                engine=engine,
                system_message_template=iter_system_message[1],
                prompt_template="{content_html}",
                additional_args={'response_name': iter_system_message[0]}
            )
        )
        message_test_configs.append(
            create_config_dict_func(
                get_response_func=get_response_func,
                rate_limiter=RateLimiter(50000),
                engine=engine,
                system_message_template="",
                prompt_template="{content_html}" + f"""\n\n""" + iter_system_message[1],
                additional_args={'response_name': "nosm_"+iter_system_message[0]}
            )
        )
    return message_test_configs

## Create configs and run tests

In [22]:
#gpt configs
gpt3_prompt_testing_configs = create_message_test_configs(system_messages_list, get_response_openai, "gpt-3.5-turbo")
gpt4_prompt_testing_configs = create_message_test_configs(system_messages_list, get_response_openai, 'gpt-4-turbo-preview')

#claude configs
haiku_prompt_testing_configs = create_message_test_configs(system_messages_list, get_response_anthropic, "claude-3-haiku-20240307")
opus_prompt_testing_configs = create_message_test_configs(system_messages_list, get_response_anthropic, "claude-3-opus-20240229")

#run the experiment on all the prompt configs and save to the folder
compare_request_configurations(dev_data_df, 
                               gpt3_prompt_testing_configs + gpt4_prompt_testing_configs + haiku_prompt_testing_configs + opus_prompt_testing_configs,
                               folder_path = dev_system_message_folder )

## Evaluate system prompt tests

We evaluate the system prompts below to see if thre is any significant difference between the prompts


In [7]:
wer = load("wer")
cer = load("cer")

In [14]:
raw_dev_ocr_scores = evaluate_correction_performance(dev_raw_ocr_folder , dev_transcripts, wer, cer, 'raw_ocr')

In [23]:
corrected_folder = dev_system_message_folder

performance_eval = evaluate_correction_performance_folders(corrected_folder, dev_transcripts, wer, cer)

performance_eval =  performance_eval.loc[(performance_eval['File Name']!='slug_ar02501_periodical_pc_issue_tec_06121884_page_number_25.txt'),:]

performance_eval['type'] = performance_eval['type'].str.replace("claude-3-haiku-20240307", "haiku").replace("gpt-3.5-turbo", "gpt-3.5")

performance_eval['model'] = performance_eval['type'].str.split('_').str[-1]

In [24]:
test = get_metric_error_reduction(performance_eval, raw_dev_ocr_scores)

In [25]:
test.loc[test['type'].str.contains('opus')].groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('CER', '50%'))

WER           CER  \
                                                     mean    50%   mean   
type                                                                      
nosm_expert_recover_publication_text_prompt_cla...  56.55  74.70 -29.86   
nosm_expert_recover_publication_prompt_claude-3...  71.64  75.47  44.57   
expert_recover_publication_prompt_claude-3-opus...  71.07  77.36  41.74   
full_context_prompt_claude-3-opus-20240229          68.79  77.39  25.49   
expert_recover_text_prompt_claude-3-opus-20240229   72.19  77.36  44.01   
basic_prompt_claude-3-opus-20240229                 72.25  77.36  41.81   
nosm_expert_recover_prompt_claude-3-opus-20240229   72.85  76.02  49.35   
nosm_expert_recover_text_prompt_claude-3-opus-2...  71.35  75.86  50.45   
nosm_basic_prompt_claude-3-opus-20240229            70.73  76.02  47.31   
expert_recover_publication_text_prompt_claude-3...  70.79  77.36  42.16   
expert_recover_prompt_claude-3-opus-20240229        63.66  78.30   4.86   
expert_basic_prompt_claude-3-opus-20240229          75.92  79.25  59.19   
nosm_expert_basic_prompt_claude-3-opus-20240229     73.73  77.59  56.30   
expert_recover_instructions_prompt_claude-3-opu...  73.67  80.46  46.82   
nosm_full_context_prompt_claude-3-opus-20240229     73.42  80.46  58.95   
nosm_expert_recover_instructions_prompt_claude-...  73.95  80.46  55.57   

                                                          lev_dist         
                                                      50%     mean    50%  
type                                                                       
nosm_expert_recover_publication_text_prompt_cla...  43.62   -29.53  45.00  
nosm_expert_recover_publication_prompt_claude-3...  49.56    43.02  48.07  
expert_recover_publication_prompt_claude-3-opus...  55.14    39.58  52.66  
full_context_prompt_claude-3-opus-20240229          55.41    25.91  56.00  
expert_recover_text_prompt_claude-3-opus-20240229   56.22    42.49  53.72  
basic_prompt_claude-3-opus-20240229                 58.38    40.48  55.60  
nosm_expert_recover_prompt_claude-3-opus-20240229   58.54    47.96  53.88  
nosm_expert_recover_text_prompt_claude-3-opus-2...  58.71    49.13  58.31  
nosm_basic_prompt_claude-3-opus-20240229            59.73    45.54  49.76  
expert_recover_publication_text_prompt_claude-3...  60.00    40.97  55.60  
expert_recover_prompt_claude-3-opus-20240229        63.23     5.51  55.60  
expert_basic_prompt_claude-3-opus-20240229          64.39    56.97  61.79  
nosm_expert_basic_prompt_claude-3-opus-20240229     64.86    54.26  58.99  
expert_recover_instructions_prompt_claude-3-opu...  68.01    45.19  62.87  
nosm_full_context_prompt_claude-3-opus-20240229     68.78    57.84  66.36  
nosm_expert_recover_instructions_prompt_claude-...  69.66    53.61  61.94

In [26]:
performance_eval2 = performance_eval.copy()
performance_eval2['type'] = performance_eval2['type'].str.replace("claude-3-haiku-20240307", "haiku").replace("gpt-3.5-turbo", "gpt-3.5")
performance_eval2['model'] = performance_eval2['type'].str.split('_').str[-1]
performance_eval2 = performance_eval2.loc[performance_eval2['model'].str.contains('gpt-4')]
performance_eval2.drop(columns = 'File Name').groupby(['type', 'model']).describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER  \
                                                                        mean   
type                                               model                       
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview  0.23   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview  0.23   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview  0.24   
expert_recover_instructions_prompt_gpt-4-turbo-... gpt-4-turbo-preview  0.23   
expert_recover_publication_prompt_gpt-4-turbo-p... gpt-4-turbo-preview  0.26   
nosm_expert_recover_prompt_gpt-4-turbo-preview     gpt-4-turbo-preview  0.25   
expert_recover_prompt_gpt-4-turbo-preview          gpt-4-turbo-preview  0.26   
nosm_expert_recover_text_prompt_gpt-4-turbo-pre... gpt-4-turbo-preview  0.25   
expert_recover_publication_text_prompt_gpt-4-tu... gpt-4-turbo-preview  0.27   
expert_recover_text_prompt_gpt-4-turbo-preview     gpt-4-turbo-preview  0.24   
basic_prompt_gpt-4-turbo-preview                   gpt-4-turbo-preview  0.24   
expert_basic_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview  0.24   
nosm_expert_basic_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview  0.24   
nosm_expert_recover_publication_text_prompt_gpt... gpt-4-turbo-preview  0.35   
nosm_basic_prompt_gpt-4-turbo-preview              gpt-4-turbo-preview  0.34   
nosm_expert_recover_publication_prompt_gpt-4-tu... gpt-4-turbo-preview  0.30   

                                                                              \
                                                                         50%   
type                                               model                       
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview  0.13   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview  0.15   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview  0.16   
expert_recover_instructions_prompt_gpt-4-turbo-... gpt-4-turbo-preview  0.14   
expert_recover_publication_prompt_gpt-4-turbo-p... gpt-4-turbo-preview  0.16   
nosm_expert_recover_prompt_gpt-4-turbo-preview     gpt-4-turbo-preview  0.16   
expert_recover_prompt_gpt-4-turbo-preview          gpt-4-turbo-preview  0.21   
nosm_expert_recover_text_prompt_gpt-4-turbo-pre... gpt-4-turbo-preview  0.14   
expert_recover_publication_text_prompt_gpt-4-tu... gpt-4-turbo-preview  0.14   
expert_recover_text_prompt_gpt-4-turbo-preview     gpt-4-turbo-preview  0.14   
basic_prompt_gpt-4-turbo-preview                   gpt-4-turbo-preview  0.18   
expert_basic_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview  0.18   
nosm_expert_basic_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview  0.14   
nosm_expert_recover_publication_text_prompt_gpt... gpt-4-turbo-preview  0.16   
nosm_basic_prompt_gpt-4-turbo-preview              gpt-4-turbo-preview  0.24   
nosm_expert_recover_publication_prompt_gpt-4-tu... gpt-4-turbo-preview  0.14   

                                                                         CER  \
                                                                        mean   
type                                               model                       
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview  0.13   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview  0.13   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview  0.14   
expert_recover_instructions_prompt_gpt-4-turbo-... gpt-4-turbo-preview  0.13   
expert_recover_publication_prompt_gpt-4-turbo-p... gpt-4-turbo-preview  0.15   
nosm_expert_recover_prompt_gpt-4-turbo-preview     gpt-4-turbo-preview  0.15   
expert_recover_prompt_gpt-4-turbo-preview          gpt-4-turbo-preview  0.15   
nosm_expert_recover_text_prompt_gpt-4-turbo-pre... gpt-4-turbo-preview  0.14   
expert_recover_publication_text_prompt_gpt-4-tu... gpt-4-turbo-preview  0.15   
expert_recover_text_prompt_gpt-4-turbo-preview     gpt-4-turbo-preview 

## Conclusions of the prompt test

It appears that the placing the prompt after the text instead of using the system prompt gives the best results. However, the prompts did give significantlty different performance.  I think that perhaps using the `full_context_prompt` and the `expert_recover_publication_prompt` with no system message and the prompt after the text may be the best option. This will require twice as much compute as I was planning to use.

# Evaluating models on the test set

Having identified two different prompts and that the prompts appear to work better when places after the text we can now compare the different models


The below code creates the basic configuration dictionaries for each model and then fills in the with the two different prompt messages creating a single list of all basic prompt/model configurations. It then calls all the LLM's and saves the results.
This works in series so takes a while.

## Create API configurations

In [28]:
#Create the prompt/system message using the best performing from the previous section

full_prompt = "{content_html}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. The text is from an english newspaper in the 1800's. The text may be an advert or article and may be missing the beggining or end. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

instruct_prompt = "{content_html}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

boros_basic  = "{content_html}"+f""" \n \n """ +"Correct the text"

boros_complex  ="{content_html}"+f""" \n \n """ + f"""Please assist with reviewing and correcting errors in texts produced by automatic transcription (OCR) of historical documents.
Your task is to carefully examine the following text and correct any mistakes introduced by the OCR software. 
Do not write anything else than the corrected text."""

In [47]:
groq_alt_endpoint = {'alt_endpoint':{'base_url':'https://api.groq.com/openai/v1',
                     'api_key':os.getenv("GROQ_API_KEY")}}

basic_model_configs = pd.DataFrame({
    'get_response_func': [get_response_openai, get_response_openai, get_response_anthropic, get_response_anthropic, 
                          get_response_openai, get_response_openai, get_response_openai], 
    'engine': ['gpt-3.5-turbo', 'gpt-4-turbo-preview', "claude-3-haiku-20240307", "claude-3-opus-20240229", 
               'mixtral-8x7b-32768', 'llama2-70b-4096', 'gemma-7b-it'],
    'rate_limit':[160e3, 80e3, 100e3, 40e3, 18e3, 30e3, 30e3],
    'additional_args': [
        {}, {}, {}, {}, 
        groq_alt_endpoint, 
        groq_alt_endpoint, 
        groq_alt_endpoint
    ]
})

base_model_configs = generate_model_configs(basic_model_configs, full_prompt, 'full')
nosm_model_configs = generate_model_configs(basic_model_configs, instruct_prompt, 'instruct')

boros_configs = [
    (get_response_openai, 'gpt-4-turbo-preview', boros_complex, "boros_complex_"),
    (get_response_openai, 'gpt-4-turbo-preview', boros_basic, "boros_basic_"),
    (get_response_anthropic, "claude-3-opus-20240229", boros_complex, "boros_complex_")
]

boros_list = [
    create_config_dict_func(
        get_response_func=config[0],
        rate_limiter=RateLimiter(80e3),
        engine=config[1],
        system_message_template="",
        prompt_template=config[2],
        additional_args={"response_name": config[3]}
    )
    for config in boros_configs
]

model_configs = base_model_configs + nosm_model_configs + boros_list



## Perform all API calls

The below section is what actually calls the API, the code points to the folders where the raw OCR is and provides a path to where the corrected text should be stored.

### NCSE

In [40]:
corrected_folder = 'data/transcription_returned_ocr/corrected_folder'


#
# This naming business needs to be cleaned up so the actual article ID is used. until then just have the following mess
#
# 

test_data_new = pd.read_csv(os.path.join(dev_data_folder,'transcription_raw_ocr.csv'))
test_data_new = test_data_new.loc[test_data_new ['file_name'].isin(files_to_df_func(dev_raw_ocr_folder )['file_name'])]

#used on the original devset
#compare_request_configurations(dev_data_df, model_configs, folder_path='./data/dev_corrected_base')
#This goes through the list of articles that have been transcribed, checks to see if there is a corrected version and if not generates it
compare_request_configurations(test_data_new, model_configs, folder_path=corrected_folder)

### Sydney Morning Herald

### Chronicalling America

## Evaluate the prompts across all models

On the smaller models, Full is worse than instruct on the larger models the reverse. Maybe this is related to ability to 'focus' or hold isntructions in memory?

In [32]:
corrected_folder = dev_system_message_folder 

performance_eval =  evaluate_correction_performance_folders(corrected_folder, dev_transcripts, wer, cer)

performance_eval =  performance_eval.loc[(performance_eval['File Name']!='slug_ar02501_periodical_pc_issue_tec_06121884_page_number_25.txt') &
                     (performance_eval['type']!='gpt3_boros_blank_gpt-3.5-turbo'),:]

In [34]:
error_reduction = get_metric_error_reduction(performance_eval, raw_dev_ocr_scores)


error_reduction.to_csv('data/analysis/error_reduction.csv', index=False)


error_reduction.groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER            CER  \
                                                     mean    50%    mean   
type                                                                       
expert_recover_text_prompt_claude-3-haiku-20240307  -3.36  13.64 -296.74   
nosm_expert_recover_publication_text_prompt_cla...   4.55  18.18 -294.91   
expert_recover_publication_text_prompt_claude-3...  17.73  35.45 -209.70   
nosm_expert_recover_text_prompt_claude-3-haiku-...   1.70  11.65 -297.54   
expert_recover_prompt_claude-3-haiku-20240307       39.84  48.11  -90.64   
...                                                   ...    ...     ...   
nosm_full_context_prompt_gpt-4-turbo-preview        73.21  77.19   54.38   
expert_recover_instructions_prompt_claude-3-opu...  73.67  80.46   46.82   
nosm_expert_basic_prompt_gpt-4-turbo-preview        73.61  77.59   55.55   
nosm_expert_recover_prompt_gpt-4-turbo-preview      72.03  75.86   49.27   
nosm_full_context_prompt_claude-3-opus-20240229     73.42  80.46   58.95   

                                                           lev_dist          
                                                       50%     mean     50%  
type                                                                         
expert_recover_text_prompt_claude-3-haiku-20240307 -197.56  -282.00 -177.78  
nosm_expert_recover_publication_text_prompt_cla... -152.94  -279.84 -144.31  
expert_recover_publication_text_prompt_claude-3... -115.22  -198.48 -110.20  
nosm_expert_recover_text_prompt_claude-3-haiku-... -107.24  -280.89 -102.16  
expert_recover_prompt_claude-3-haiku-20240307       -39.82   -85.44  -36.21  
...                                                    ...      ...     ...  
nosm_full_context_prompt_gpt-4-turbo-preview         63.78    52.09   62.23  
expert_recover_instructions_prompt_claude-3-opu...   68.01    45.19   62.87  
nosm_expert_basic_prompt_gpt-4-turbo-preview         62.80    53.41   63.64  
nosm_expert_recover_prompt_gpt-4-turbo-preview       65.55    47.03   63.79  
nosm_full_context_prompt_claude-3-opus-20240229      68.78    57.84   66.36  

[64 rows x 6 columns]

In [41]:
performance_eval2 = performance_eval.copy()
performance_eval2['type'] = performance_eval2['type'].str.replace("claude-3-haiku-20240307", "haiku").replace("gpt-3.5-turbo", "gpt-3.5")
performance_eval2['model'] = performance_eval2['type'].str.split('_').str[-1]

#The below line allows you to look at an individual model
#performance_eval2 = performance_eval2.loc[performance_eval2['model'].str.contains('gpt-4')]

performance_eval2.drop(columns = 'File Name').groupby(['type', 'model']).describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER  \
                                                                           mean   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.23   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.23   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.24   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.24   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.25   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.57   
expert_recover_publication_text_prompt_haiku       haiku                   0.71   
nosm_expert_recover_text_prompt_haiku              haiku                   0.81   
expert_recover_text_prompt_haiku                   haiku                   0.83   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.81   

                                                                                 \
                                                                            50%   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.13   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.15   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.12   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.16   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.11   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.51   
expert_recover_publication_text_prompt_haiku       haiku                   0.51   
nosm_expert_recover_text_prompt_haiku              haiku                   0.73   
expert_recover_text_prompt_haiku                   haiku                   0.90   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.68   

                                                                            CER  \
                                                                           mean   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.13   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.13   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.14   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.14   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.15   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.44   
expert_recover_publication_text_prompt_haiku       haiku                   0.58   
nosm_expert_recover_text_prompt_haiku              haiku                   0.65   
expert_recover_text_prompt_haiku                   haiku                   0.68   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.66   

                                                                                 \
                                                                            50%   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.06   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.06   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.05   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.07   
nosm_f

# Test on the test set
confusing but you know what I mean, This is jsut to make sure it all works as expected

In [19]:
transcribed_files = 'data/transcription_returned_ocr/transcription_files'

corrected_folder = 'data/transcription_returned_ocr/corrected_folder'

raw_folder = "/home/jonno/redigitalize/data/transcription_raw_ocr"

performance_eval =  evaluate_correction_performance_folders(corrected_folder, transcribed_files, wer, cer)

performance_eval =  performance_eval.loc[(performance_eval['File Name']!='slug_ar02501_periodical_pc_issue_tec_06121884_page_number_25.txt') &
                     (performance_eval['type']!='gpt3_boros_blank_gpt-3.5-turbo'),:]

In [20]:
performance_eval.describe()

,WER,CER,lev_dist
count,324.000000,324.000000,324.000000
mean,0.427249,0.320615,674.836420
std,1.404056,1.181777,1563.578346
min,0.000000,0.000000,1.000000
25%,0.047231,0.017892,55.750000
50%,0.160026,0.080230,130.000000
75%,0.383333,0.224831,446.000000
max,17.497006,13.418564,15369.000000


In [21]:
raw_dev_ocr_scores = evaluate_correction_performance(raw_folder, transcribed_files, wer, cer, 'raw_ocr')

error_reduction_df = get_metric_error_reduction(performance_eval, raw_dev_ocr_scores)

error_reduction_df.groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER             CER        lev_dist  \
                                         mean    50%     mean    50%     mean   
type                                                                            
full__gemma-7b-it                       10.12  27.19  -253.41 -60.56  -249.00   
instruct__gemma-7b-it                   24.95  39.52  -172.24 -37.54  -168.29   
full__mixtral-8x7b-32768                36.60  43.54  -133.47 -37.87  -134.53   
instruct__mixtral-8x7b-32768            44.84  55.42   -81.32 -27.66   -79.69   
full__claude-3-haiku-20240307           21.36  57.17  -295.09  -4.68  -285.72   
instruct__llama2-70b-4096              -15.26  42.98  -374.12  -0.88  -356.15   
instruct__claude-3-haiku-20240307       40.83  57.99  -189.58   9.43  -183.53   
full__llama2-70b-4096                 -161.34  58.97  -991.54  15.29  -909.02   
full__gpt-3.5-turbo                     59.10  70.00   -45.12  39.74   -42.64   
instruct__gpt-3.5-turbo                 56.68  75.36   -17.02  48.57   -14.90   
boros_complex__claude-3-opus-20240229   25.70  81.12  -192.53  58.70  -173.41   
full__gpt-4-turbo-preview               77.58  80.38    59.46  62.84    57.26   
boros_basic__gpt-4-turbo-preview        76.37  77.21    55.96  62.65    54.33   
instruct__gpt-4-turbo-preview           78.17  79.38    61.62  66.50    59.28   
boros_complex__gpt-4-turbo-preview      79.80  84.22    66.96  70.37    64.79   
full__claude-3-opus-20240229            -4.45  82.24  -375.41  78.35  -335.13   
claude_temp_claude-3-opus-20240229      22.89  82.24  -231.82  78.64  -205.53   
instruct__claude-3-opus-20240229      -136.72  79.25 -1519.07  78.89 -1449.43   

                                              
                                         50%  
type                                          
full__gemma-7b-it                     -53.34  
instruct__gemma-7b-it                 -35.61  
full__mixtral-8x7b-32768              -33.88  
instruct__mixtral-8x7b-32768          -28.71  
full__claude-3-haiku-20240307          -2.99  
instruct__llama2-70b-4096              -0.36  
instruct__claude-3-haiku-20240307       9.42  
full__llama2-70b-4096                  14.65  
full__gpt-3.5-turbo                    39.18  
instruct__gpt-3.5-turbo                47.85  
boros_complex__claude-3-opus-20240229  58.29  
full__gpt-4-turbo-preview              61.85  
boros_basic__gpt-4-turbo-preview       62.33  
instruct__gpt-4-turbo-preview          63.97  
boros_complex__gpt-4-turbo-preview     67.48  
full__claude-3-opus-20240229           75.89  
claude_temp_claude-3-opus-20240229     76.17  
instruct__claude-3-opus-20240229       76.44

# Evaluate performance on downstream tasks

In [50]:
(450e6/9e3)/60/24

34.72222222222222

In [53]:
500e6 /(760*3600)/24

7.614522417153996

In [23]:
ner_model = "Gladiator/microsoft-deberta-v3-large_ner_conll2003"#"dslim/bert-base-NER"

transcribed_data_set_df = files_to_df_func(transcribed_files)
raw_data_set_df = files_to_df_func(raw_folder)

raw_data_set_df = raw_data_set_df.loc[raw_data_set_df['file_name'].isin(transcribed_data_set_df['file_name'])]

tokenizer = AutoTokenizer.from_pretrained(ner_model)

model = AutoModelForTokenClassification.from_pretrained(ner_model)
def preprocess_text(text):
    # Example preprocessing: substitute multiple whitespaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text

perform_ner = pipeline("ner", model=model, tokenizer=tokenizer)


def perform_ner_on_text(text, ner_pipeline):
    # Preprocess the text first
    preprocessed_text = preprocess_text(text)
    # Then pass the preprocessed text to the NER pipeline
    return ner_pipeline(preprocessed_text)

transcribed_data_set_df['ner_results'] = transcribed_data_set_df['content'].apply(perform_ner_on_text, ner_pipeline=perform_ner)

raw_data_set_df['ner_results_raw'] = raw_data_set_df['content'].apply(perform_ner_on_text, ner_pipeline=perform_ner)

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [24]:
#load on the transcribed data... this should be done within a loop as there are many models to test
LM_corrected_df = files_to_df_func(os.path.join(corrected_folder, 'claude_temp_claude-3-opus-20240229'))
temp_ner = transcribed_data_set_df.copy().merge(LM_corrected_df.loc[:,['content', 'file_name'] ].rename(columns={'content':'content_corrected'}), 
                                                on = 'file_name')

temp_ner['ner_results_corrected'] = temp_ner['content_corrected'].apply(perform_ner_on_text, ner_pipeline=perform_ner)

temp_ner['cosine_sim'] = temp_ner.apply(lambda row: calculate_entity_similarity(row['ner_results_corrected'],row['ner_results']), axis = 1)


print("Median cosine similarity:", temp_ner['cosine_sim'].median().round(2))

Median cosine similarity: 0.92


In [26]:
results, results_by_tag = evaluate_ner(temp_ner, 'ner_results', 'ner_results_corrected')

print(results)
print(results_by_tag)

KeyError: 'ner_results_corrected'

In [27]:
temp_ner = transcribed_data_set_df.copy().merge(raw_data_set_df.loc[:,['ner_results_raw', 'file_name'] ].rename(columns={'content':'content_corrected'}), 
                                                on = 'file_name')

temp_ner['cosine_sim'] = temp_ner.apply(lambda row: calculate_entity_similarity(row['ner_results_raw'],row['ner_results']), axis = 1)

print("Median cosine similarity:", temp_ner['cosine_sim'].median().round(2))

results, results_by_tag = evaluate_ner(temp_ner, 'ner_results', 'ner_results_raw')

print(results)
print(results_by_tag)

Median cosine similarity: 0.74
{'ent_type': {'correct': 97, 'incorrect': 63, 'partial': 0, 'missed': 588, 'spurious': 545, 'possible': 748, 'actual': 705, 'precision': 0.1375886524822695, 'recall': 0.12967914438502673, 'f1': 0.1335168616655196}, 'partial': {'correct': 16, 'incorrect': 0, 'partial': 144, 'missed': 588, 'spurious': 545, 'possible': 748, 'actual': 705, 'precision': 0.12482269503546099, 'recall': 0.11764705882352941, 'f1': 0.12112869924294564}, 'strict': {'correct': 14, 'incorrect': 146, 'partial': 0, 'missed': 588, 'spurious': 545, 'possible': 748, 'actual': 705, 'precision': 0.019858156028368795, 'recall': 0.01871657754010695, 'f1': 0.019270474879559532}, 'exact': {'correct': 16, 'incorrect': 144, 'partial': 0, 'missed': 588, 'spurious': 545, 'possible': 748, 'actual': 705, 'precision': 0.02269503546099291, 'recall': 0.0213903743315508, 'f1': 0.022023399862353753}}
{'B-LOC': {'ent_type': {'correct': 28, 'incorrect': 15, 'partial': 0, 'missed': 84, 'spurious': 78, 'possib

In [28]:
temp_ner['ner_results'][11]#['content'][10]

[{'entity': 'B-ORG',
  'score': 0.95968384,
  'index': 19,
  'word': '▁Review',
  'start': 89,
  'end': 96}]

# Demonstrating context leveraging

The below texts can be used to demonstrate that LMs use context leveraging and that they can self-contextualise from the task itself. The words in the parenthesis are optional contxt

"""Please correct the below sentences containing OCR errors

My dog has no ***

*** did *** **** cross ***"""

In [2]:



response = get_response_openai(
    """Please correct the below sentences containing OCR errors

My dog has no ***

*** did *** **** cross ***""", 
"", RateLimiter(40000), 
engine="gpt-4-turbo-preview", 
max_tokens = 500)

2024-04-16 06:58:02 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [116]:
from openai import OpenAI

client = OpenAI()

In [210]:

dog_nose = """My dog *** *** nose. 
How *** he ***? """

dog_nose = """My dog has no  ***"""

chicken_cross = "*** did *** **** cross ***"

chicken_cross = "why did the **** *** ***"


prompt_dog =f"""Please correct the below sentences containing OCR errors

{dog_nose}

"""

prompt_dog_joke =f"""Please correct the below sentences containing OCR errors, the sentence is part of a popular joke

{dog_nose}

"""

check_prompt_dog = """The below text surrounded by triple colons is the response of a language model to a task, did it correctly identify the sentence 'my dog has no nose', Answer 1 or 0
:::
{answer}
:::

"""

prompt_chicken =f"""Please correct the below sentences containing OCR errors

{chicken_cross}

"""

prompt_chicken_joke =f"""Please correct the below sentences containing OCR errors, the sentence is part of a popular joke

{chicken_cross}

"""

check_prompt_chicken = """The below text surrounded by triple colons is the response of a language model to a task, did it correctly identify the sentence 'why did the chicken cross the road', Answer 1 or 0
:::
{answer}
:::

"""


prompt_combined =f"""Please correct the below sentences containing OCR errors

{dog_nose}

{chicken_cross}
"""

prompt_combined_joke =f"""Please correct the below sentences containing OCR errors, the sentences are part of a popular jokes

{dog_nose}

{chicken_cross}
"""

check_combined = """The below text surrounded by triple colons is the response of a language model to a task, did it correctly identify both the sentences 'my dog has no nose' and 'why did the chicken cross the road', Answer 1 if both identified or 0 otherwise
:::
{answer}
:::

"""


In [232]:
context_check = pd.DataFrame([
             {'prompt': prompt_dog, 'check':check_prompt_dog, 'context':'no', 'joke':'dog' },
             {'prompt': prompt_dog_joke, 'check':check_prompt_dog, 'context':'yes', 'joke':'dog' },
             {'prompt': prompt_chicken, 'check':check_prompt_chicken, 'context':'no', 'joke':'chicken' },
             {'prompt': prompt_chicken_joke, 'check':check_prompt_chicken, 'context':'yes', 'joke':'chicken' },
             {'prompt': prompt_combined, 'check':check_combined, 'context':'no', 'joke':'combined' },
             {'prompt': prompt_combined_joke, 'check':check_combined, 'context':'yes', 'joke':'combined' }
            ])


context_results =[]
for index, row in context_check.iterrows():

    result = repeat_prompt_experiment(row['prompt'] , row['check'] , client, repititions = 100)

    result['context'] = row['context']
    result['joke'] = row['joke']

    context_results.append(result)


context_results = pd.concat(context_results, ignore_index=True)

#context_results.to_csv("does_LM_use_context_results.csv")

2024-04-16 10:35:25 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:26 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:27 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:27 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:28 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:29 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:30 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:31 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:32 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 